In [1]:
import pandas as pd
import numpy as np
import shutil
import os
import datetime

elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\CSIRO_GBR_ElementMapperBundledOutlets.csv'
#Names from element mapper
regID = 'RegID'
riverCol = 'Major River'
elementCol = 'Network Element'

#Use this to write out (or not) individual outlet files
WriteIndividual = False

#GBRF Scenarios
#simMapperCSV = r'P:\projects\eReefs\GBRF_SpeedScenarios\GBRF_ScenarioExtension_ResultsMapperFiltered.csv'

##All sediment strategies together
#simMapperCSV = r'P:\projects\eReefs\GBRF_SpeedScenarios\GBRF_ScenarioExtension_ResultsMapperFiltered_FSED_ALL.csv'

#Standard eReefs baseline/pre-dev
simMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\CSIRO_eReefsStandard_ResultsMapper_Nov2021.csv'


#Names from sim mapper, use RegID from above
basinFolderName = 'BasinFolder'
newSimFolderName = 'SIMCODE'
#Switch these for TS Dependent or EMCMonthly
existSimFolderName = 'TSDependentSubDir'
#existSimFolderName = 'EMCMonthly2021SubDir'

#Switch these for parent location of TS Dependent or EMCMonthly results
#TSDependent
# parentResultsDir = r'E:\GBRFScenarios\Results'
# addRegIdDIR = True
# outPath = r'E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet'
#Empirical
parentResultsDir = r'P:\projects\RC10_ResultsSets\Models\results'
addRegIdDIR = False
#outPath = r'E:\GBRFScenarios\CSIROAggregatedOutlets\EmpiricalEMC'
##outPath = r'P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\EmpiricalEMCNov2021'
#outPath = r'P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\EmpiricalEMCNov2021WithIndividual'
outPath = r'P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018'
#outPath = r'P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018WithIndividual'

#, 'Sediment - Coarse'

Constituents = ['Sediment - Fine', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']
#Constituents = ['Ametryn', 'S-metolachlor', 'Atrazine', 'Diuron', 'Hexazinone', 'Tebuthiuron', '24-D'
#                , 'Paraquat', 'Glyphosate', 'Chlorsulfuron', 'Diquat', 'Fluroxypyr', 'Haloxyfop', 'Imazapic'
#               , 'Imazethapyr', 'Isoxaflutole', 'MCPA', 'Metribuzin', 'Metsulfuron-methyl', 'Pendimethalin', 'Simazine'
#               , 'Terbuthylazin', 'Trifluralin', 'Aciflurofen', 'Chlorpyrifos', 'Fipronil', 'Imidacloprid', 'Prometryn', 'Triclopyr']

#regionIDs = ['BU','FI','CY','BM', 'WT', 'MW']
#,'CY':'CY Rebuild 2015'
#regionIDs = {'BU':'Burdekin Rebuild 2014','FI':'FI RC2019', 'WT':'WT_RC10'}

#regionIDs = {'BM':'Burnett Mary Rebuild 2014','BU':'Burdekin Rebuild 2014','FI':'FI RC2019',
#              'MW':'MW_RC10', 'WT':'WT_RC10'}


regionIDs = {'BM':'Burnett Mary Rebuild 2014','BU':'Burdekin Rebuild 2014','FI':'FI RC2019',
              'MW':'MW_RC10', 'WT':'WT_RC10', 'CY':'CY Rebuild 2015'}


#After fixing inflow units
#regionIDs = {'MW':'MW_RC10'}

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BurdekinElementMapper.csv'
#mainPath = r'P:\projects\MichaelWarne\Models\results\Burdekin Rebuild 2014\BU_msPAF_Calibrated_503\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\Burdekin'



In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:

# ##This bit now needs to happen inside of loop
# RiverElements = {}
# elementsTable = pd.read_csv(elementMapperCSV)

# for index, row in elementsTable.iterrows():
#     if not row[riverCol] in RiverElements.keys():
#         RiverElements[row[riverCol]] = []
    
#     RiverElements[row[riverCol]].append(row[elementCol])


# #print(RiverElements)

In [4]:
allElementsTable = pd.read_csv(elementMapperCSV)

In [5]:
simDetailsDF = pd.read_csv(simMapperCSV)

In [6]:
def getResultsPathInfo(regionIDString, scenarioString):
    tsPath = ''
    if addRegIdDIR:
        tsPath = parentResultsDir + '\\' + regionIDString + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    else:
        tsPath = parentResultsDir + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    return tsPath

In [7]:
def doThisSim(thisRegID, thisBasinFolderName, thisNewSimFolderName, thisExistSimFolderName, theseRiverElements):
    
    #Going to need to set up folders
    #basinOutputStr = outPath + '\\' + thisBasinFolderName
    basinOutputStr = os.path.join(outPath, thisBasinFolderName)
    if not os.path.exists(basinOutputStr):
        os.makedirs(basinOutputStr)
        print("Made dir: " + basinOutputStr)
    
    
    simOutputStr = os.path.join(basinOutputStr, thisNewSimFolderName)
    if not os.path.exists(simOutputStr):
        os.makedirs(simOutputStr)
        print("Made dir: " + simOutputStr)    
    
    #Set up the totals first
    RiverElements = {}
    # elementsTable = pd.read_csv(elementMapperCSV)

    for index, row in theseRiverElements.iterrows():
         if not row[riverCol] in RiverElements.keys():
            RiverElements[row[riverCol]] = []
    
         RiverElements[row[riverCol]].append(row[elementCol])
    
    riverTotals = {}
    for theRiver in RiverElements.keys():
        riverTotals[theRiver] = pd.DataFrame()

    #Now data
    mainPath = getResultsPathInfo(thisRegID, thisExistSimFolderName)

    theFlowsDIR = os.path.join(mainPath,'Flows')
    for theRiver in RiverElements.keys():
        #We'll want flows first
        eleCount = 0
        flowFrame = pd.DataFrame()
        outputFields = ['Date']
        for theElement in RiverElements[theRiver]:
            #This will stope Outlet Node1 matching Outle Node10, Outlet Node11
            theElementUnder = theElement + '_'
            for fname in os.listdir(theFlowsDIR):
                if theElementUnder in fname:
                    #print("For this river: " + theRiver + " we have this file: " + fname)
                    flowFilePath = os.path.join(theFlowsDIR,fname)
                    flowData = pd.read_csv(flowFilePath)
                    flowData.columns = ['Date', theElement]
                    #riverTotals[theRiver].columns.drop(theElement)
                    outputFields.append(theElement)
                    if eleCount == 0:
                        #Get the structure for our totals first
                        riverTotals[theRiver] = flowData
                        
                        #Dropping this means we lose reference to the first (or only) named Element.
                        #riverTotals[theRiver].drop(theElement, axis=1, inplace=True)
                        #Now process stuff
                        flowFrame = flowData
                        #flowFrame.columns = ['Date', theElement]
                        #flowFrame['Date'] = flowData['Date']
                        #flowFrame[theElement] = flowData.iloc[:1].values#This sould give the 2nd column
                    else:
                        #Join...??
                        #flowFrame[theElement] = flowData.iloc[:1].values
                        #flowData will have flow col named after element, so won't be duplicated
                        flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
                
                    eleCount += 1
        #Totals
        #The .iloc[:, 1:] should exclude our first col, Date, from the sum
        #flowFrame['Flow_cumecs'] = flowFrame.iloc[:, 1:].sum(axis=1)
        flowFrame['Flow_cumecs'] = flowFrame.sum(axis=1)
        #Write out flow
        theOutFlowFile = os.path.join(simOutputStr, "Flow " + theRiver + " cubicmetrespersecond.csv")
        if WriteIndividual:
            flowFrame.to_csv(theOutFlowFile, index=False)
        
        #Need to join flow Total to RiverTotals
        #This should be using ONLY 'Date' and 'Flow_cumecs', thereby dropping theElement from riverTotals... but is it???
        #riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])
        #When doing Flow, we're creating these, so we should never hit the ELSE here, and vice versa foWQ
        if len(riverTotals[theRiver].columns):
            #print("We were empty doing: " + theRiver)
            riverTotals[theRiver] = flowFrame[['Date', 'Flow_cumecs']]
        else:
            #print("NOOOOOOTTTTT empty doing: " + theRiver)
            riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])

    #Now water quality
    for theWQBit in Constituents:
        wqPath = os.path.join(mainPath, theWQBit)
        if not os.path.exists(wqPath):
            continue
    
        for theRiver in RiverElements.keys():
            #We'll want flows first
            eleCount = 0
            wqFrame = pd.DataFrame()
            outputFields = ['Date']
            for theElement in RiverElements[theRiver]:
                theElementUnder = theElement + '_'
                for fname in os.listdir(wqPath):
                    if theElementUnder in fname:
                        #print("For this river: " + theRiver + " we have this file: " + fname)
                        wqFilePath = os.path.join(wqPath,fname)
                        wqData = pd.read_csv(wqFilePath)
                        wqData.columns = ['Date', theElement]
                        outputFields.append(theElement)
                        if eleCount == 0:
                            wqFrame = wqData
                        else:
                            wqFrame = pd.merge(wqFrame, wqData, how='left', left_on=['Date'], right_on=['Date'])
                
                        eleCount += 1
            #Totals
            #wqFrame[theRiver + '_kg'] = wqFrame.iloc[:, 1:].sum(axis=1)
            wqFrame[theRiver + '_kg'] = wqFrame.sum(axis=1)
            #Write out flow
            theOutFlowFile = os.path.join(simOutputStr, theWQBit + " " + theRiver + " kilograms.csv")
            if WriteIndividual:
                wqFrame.to_csv(theOutFlowFile, index=False)
            
            #Join to totals
            #rename constituent total first
            #print('theRiver: ' + theRiver)
            #print('theWQBit: ' + theWQBit)
            wqFrame.rename(columns={theRiver + '_kg':theWQBit + '_kg'}, inplace=True)
            riverTotals[theRiver] = pd.merge(riverTotals[theRiver], wqFrame[['Date', theWQBit + '_kg']], how='left', left_on=['Date'], right_on=['Date'])


    #Now write totals files
    for theRiver in RiverElements.keys():
        theOutTotalsFile = os.path.join(simOutputStr, theRiver + " Totals.csv")
        riverTotals[theRiver].to_csv(theOutTotalsFile, index=False)


    print("Done " + thisNewSimFolderName + " at " + str(datetime.datetime.now()))

In [8]:
#This is where we will loop through sims
for index, row in simDetailsDF.iterrows():
    if row[regID] in regionIDs.keys():
        print("Doing this simulation: " + row[newSimFolderName] + " at " + str(datetime.datetime.now()))
        #Filter out the river elements for this region
        filteredElements = allElementsTable.loc[allElementsTable[regID] == row[regID]]
        doThisSim(row[regID], row[basinFolderName], row[newSimFolderName], row[existSimFolderName], filteredElements)

print("All finished at " + str(datetime.datetime.now()))

Doing this simulation: WTBASE at 2022-06-01 16:05:33.740173


<ipython-input-7-deaaa3986cb0>:71: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  flowFrame['Flow_cumecs'] = flowFrame.sum(axis=1)
<ipython-input-7-deaaa3986cb0>:65: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TR03_x'} in the result is deprecated and will raise a MergeError in a future version.
  flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
<ipython-input-7-deaaa3986cb0>:65: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TR02_x'} in the result is deprecated and will raise a MergeError in a future version.
  flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
<ipython-input-7-deaaa3986cb0>:116: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprec

Done WTBASE at 2022-06-01 16:05:49.385992
Doing this simulation: WTPREDEV at 2022-06-01 16:05:49.385992
Done WTPREDEV at 2022-06-01 16:06:10.460181
Doing this simulation: MWBASE at 2022-06-01 16:06:10.460181
Done MWBASE at 2022-06-01 16:06:31.511214
Doing this simulation: MWPREDEV at 2022-06-01 16:06:31.520214
Made dir: P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018\MackayWhitsundays\MWPREDEV
Done MWPREDEV at 2022-06-01 16:06:59.517227
Doing this simulation: CYBASE at 2022-06-01 16:06:59.532857
Made dir: P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018\CapeYork
Made dir: P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018\CapeYork\CYBASE
Done CYBASE at 2022-06-01 16:07:42.854733
Doing this simulation: CYPREDEV at 2022-06-01 16:07:42.864826
Made dir: P:\projects\RC10_ResultsSets\CSIROAggregatedOutlets\ReportCard2018\CapeYork\CYPREDEV
Done CYPREDEV at 2022-06-01 16:08:26.650074
All finished at 2022-06-01 16:08:26.650074
